*Арешин Станислав Олегович*

**Лабораторная работа №3**

**Проверка статистических гипотез для количественных признаков**

# Реализация

## Генерация выборок


In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats

In [2]:
def gen_gaussian_lin_mono(mu, sigma, n):
    x = np.random.normal(mu, sigma, n)
    z = np.random.normal(mu, sigma, n)
    y = 0.6 * x + 0.4 * z
    return x, y

def gen_notgaussian_notlin_mono(mu, sigma, n):
    x = np.random.normal(mu, sigma, n)
    z = np.random.normal(mu*2, sigma*10, n)
    y = np.exp(2*x) + z
    return x, y

def gen_notgaussian_notlin_notmono(mu, sigma, n):
    x = np.random.normal(mu, sigma, n)
    z = np.random.normal(mu*2, sigma*2, n)
    y = 2 * x ** 2 + z ** 2
    return x, y

## Выборочный коэффициент корреляции

Проверка зависимости двух нормально распределённых признаков

In [3]:
def Sample_corr_test(x, y, alpha = 0.05):
    # флаг, который указывает, принимаем H0 или отвергаем : 1-отвергаем, 0 - принимаем
    complete = 0
    # кол-во элементов в выборке
    n = len(x)
    # вычисялем выборочное среднее выборок
    x_mean = x.mean()
    y_mean = y.mean()
    # вычисляем выборочный коэф корреляции
    r = (1 / n * np.sum((x - x_mean) * (y - y_mean))) / \
    (1 / n * np.sum((x - x_mean) ** 2) * 1 / n * np.sum((y - y_mean) ** 2)) ** 0.5
    # вычисляем статистику критерия t_value
    t_value = n ** 0.5 * r
    # вычисляем p-value
    p_value = np.min(np.array([2 * stats.norm.cdf(t_value), 2 - 2 * stats.norm.cdf(t_value)]), axis=0)
    # принимаем H0 или отвергаем
    if (p_value > alpha / 2) and (p_value < (1 - alpha / 2)):
        complete = 0
    else:
        complete = 1
    return r, t_value, p_value, complete

In [4]:
# реализованный

x1, y1 = gen_gaussian_lin_mono(2, 10, 100)
Sample_corr_test(x1, y1)

(0.7827665818936483, 7.827665818936484, 4.884981308350689e-15, 1)

In [5]:
# проверка scipy

stats.pearsonr(x1, y1)

(0.7827665818936483, 6.636176372467055e-22)

In [6]:
# реализованный

x2, y2 = gen_notgaussian_notlin_mono(2, 10, 100)
Sample_corr_test(x2, y2)

(0.2661429758685755, 2.661429758685755, 0.007780957370092656, 1)

In [7]:
# проверка scipy

stats.pearsonr(x2, y2)

(0.26614297586857544, 0.007441286427588205)

In [8]:
# реализованный

x3, y3 = gen_notgaussian_notlin_notmono(2, 10, 100)
Sample_corr_test(x3, y3)

(0.370143598972418, 3.70143598972418, 0.00021438274235618948, 1)

In [9]:
# проверка scipy

stats.pearsonr(x3, y3)

(0.3701435989724181, 0.0001503746570864822)

## Критерий Спирмана

Признаки ненормальны, подозревается монотонная зависимость

In [10]:
def ranging(x):
    # отсортированная выборка
    x_sorted = np.sort(x)
    # массив рангов
    R = np.array([])
    elems = np.array([])
    x_rangs =  np.array([])
    # номер итерации
    i = 1
    # вычисляем ранги
    while i <= len(x):
        # кол-во одинаковых элементов
        x_count = (x_sorted == x_sorted[i-1]).sum()
        # если элемент уникален
        if x_count == 1:
            # добавялем ранг
            R = np.append(R, i)
            # добавялем элемент
            elems = np.append(elems, x_sorted[x_sorted == x_sorted[i-1]])
        # если элемент неуникален
        else:
            # вычисляем и добавляем ранги
            for j in range(x_count):
                R = np.append(R, i + (c - 1) / 2)
                elems = np.append(elems, x_sorted[x_sorted == x_sorted[i-1]][0])
        i += x_count
    for i in range(len(x)):
        idx = np.where(elems == x[i])
        x_rangs = np.append(x_rangs, R[idx[0]])
    return x_rangs

def Spearman_test(x, y, alpha = 0.05):
    # флаг, который указывает, принимаем H0 или отвергаем : 1-отвергаем, 0 - принимаем
    complete = 0
    # кол-во элементов в выборке
    n = len(x)
    # вычисляем ранги выборок x и y
    x_rangs = ranging(x)
    y_rangs = ranging(y)
    # вычисляем ранговый коэф корреляции спирмана
    r_spearman = 1 - 6 / (n ** 3 - n) * np.sum((x_rangs - y_rangs) ** 2)
    # вычисляем статистику критерия t_value
    t_value = (n-1) ** 0.5 * r_spearman
    # вычисляем p-value
    p_value = np.min(np.array([2 * stats.norm.cdf(t_value), 2 - 2 * stats.norm.cdf(t_value)]), axis=0)
    # принимаем H0 или отвергаем
    if (p_value > alpha / 2) and (p_value < (1 - alpha / 2)):
        complete = 0
    else:
        complete = 1
    return r_spearman, t_value, p_value, complete

In [11]:
# реализованный 
    
Spearman_test(x1, y1)

(0.7742814281428143, 7.704002937870723, 1.3100631690576847e-14, 1)

In [12]:
# проверка scipy

stats.spearmanr(x1, y1)

SpearmanrResult(correlation=0.7742814281428142, pvalue=3.4702856182425166e-21)

In [13]:
# реализованный  
    
Spearman_test(x2, y2)

(0.8454485448544855, 8.412106808502857, 0.0, 1)

In [14]:
# проверка scipy

stats.spearmanr(x2, y2)

SpearmanrResult(correlation=0.8454485448544854, pvalue=1.905427836713662e-28)

In [15]:
# реализованный 

Spearman_test(x3, y3)

(0.26445844584458444, 2.6313283125210294, 0.008505182842968528, 1)

In [16]:
# проверка scipy

stats.spearmanr(x3, y3)

SpearmanrResult(correlation=0.26445844584458444, pvalue=0.007840787782760853)

## Критерий Хи-квадрат Пирсона

Признаки ненормальны, подозревается немонотонная нелинейная зависимость

In [17]:
def to_nominal(x, k):
    x_sort = np.sort(x)
    x_in_scale = np.zeros(k)
    slices = np.linspace(x_sort[0], x_sort[-1], k+1)
    for i in range(k):
        x_part = x[x >= slices[i]]
        x_count = (x_part <= slices[i+1]).sum()
        x_in_scale[i] = x_count
    return x_in_scale

def Pirson_test(x, y, coeffs = True, alpha = 0.05):
    # флаг, который указывает, принимаем H0 или отвергаем : 1-отвергаем, 0 - принимаем
    complete = 0
    # кол-во элементов в выборке
    n = len(x)
    # переводим в номинальную шкалу
    x_nom = to_nominal(x, 5)
    y_nom = to_nominal(y, 5)
    matrix = np.concatenate(([x_nom], [y_nom])) 
    chi2_value = 0
    for i in range(2):
        for j in range(len(x_nom)):
            chi2_value += (matrix[i][j] - (np.sum(matrix[i]) * np.sum(matrix[:,j]) / np.sum(matrix))) ** 2 \
            / (np.sum(matrix[i]) * np.sum(matrix[:,j]) / np.sum(matrix))
            
    # вычисляем коэффициенты
    if coeffs == True:
        pirson_coeff = (chi2_value / (np.sum(matrix) + chi2_value)) ** 0.5
        kramer_coeff = (chi2_value / (np.sum(matrix) * np.min([(len(x_nom) - 1), 1]))) ** 0.5
        chuprov_coeff =  (chi2_value / (np.sum(matrix) * ((len(x_nom) - 1)) ** 0.5)) ** 0.5
        print(f'Pirson coeff = {pirson_coeff}\n')
        print(f'Kramer coeff = {kramer_coeff}\n')
        print(f'Chuprov coeff = {chuprov_coeff}\n')
    # вычисляем p-value
    p_value = 1 - stats.chi2.cdf(chi2_value, (len(x_nom) - 1))
    # принимаем H0 или отвергаем
    if p_value > alpha:
        complete = 0
    else:
        complete = 1
    return chi2_value, p_value, complete

In [18]:
# реализованный  
    
Pirson_test(x1, y1)

Pirson coeff = 0.24919087193978878

Kramer coeff = 0.2573078030352875

Chuprov coeff = 0.1819440923784643



(13.241461100569259, 0.010154297009007496, 1)

In [19]:
# проверка scipy

from scipy.stats import chi2_contingency
x_nom = to_nominal(x1, 5)
y_nom = to_nominal(y1, 5)
chi2_contingency([x_nom, y_nom])

(13.241461100569259,
 0.010154297009007494,
 4,
 array([[ 4. , 30. , 42.5, 15.5,  8. ],
        [ 4. , 30. , 42.5, 15.5,  8. ]]))

In [20]:
# реализованный  

Pirson_test(x2, y2)

Pirson coeff = 0.6819506889765271

Kramer coeff = 0.9323932340258282

Chuprov coeff = 0.6593015785121187



(173.87142857142857, 0.0, 1)

In [21]:
# проверка scipy

from scipy.stats import chi2_contingency
x_nom = to_nominal(x2, 5)
y_nom = to_nominal(y2, 5)
chi2_contingency([x_nom, y_nom])

(173.87142857142857,
 1.5433495549008284e-36,
 4,
 array([[52.5, 15. , 19. ,  9.5,  4. ],
        [52.5, 15. , 19. ,  9.5,  4. ]]))

In [22]:
# реализованный 

Pirson_test(x3, y3)

Pirson coeff = 0.5000044465941806

Kramer coeff = 0.5773571151996598

Chuprov coeff = 0.40825313132398217



(66.66824769433465, 1.1457501614131615e-13, 1)

In [23]:
# проверка scipy

from scipy.stats import chi2_contingency
x_nom = to_nominal(x3, 5)
y_nom = to_nominal(y3, 5)
chi2_contingency([x_nom, y_nom])

(66.66824769433465,
 1.1452490058567674e-13,
 4,
 array([[44. , 23. , 23. ,  7.5,  2.5],
        [44. , 23. , 23. ,  7.5,  2.5]]))

# Тест

## Подсчёт ошибок 2-го рода

In [24]:
def beta_error_frequency(mu, sigma, n, num_experiments, f):
    Corr_test_result= []
    Spearman_test_result = []
    Pirson_test_result = []
    for i in range(num_experiments):
        x, y  =  f(mu, sigma, n)
        Corr_test_result.append(Sample_corr_test(x, y)[3])
        Spearman_test_result.append(Spearman_test(x, y)[3])
        Pirson_test_result.append(Pirson_test(x, y, coeffs = False)[2])
    Corr_test_frequency = (len(Corr_test_result) - sum(Corr_test_result)) / len(Corr_test_result)
    Spearman_test_frequency = (len(Spearman_test_result) - sum(Spearman_test_result)) / len(Spearman_test_result)
    Pirson_test_frequency = (len(Pirson_test_result) - sum(Pirson_test_result)) / len(Pirson_test_result)
    return Corr_test_frequency,  Spearman_test_frequency, Pirson_test_frequency

## Функция отрисовки графиков

In [394]:
def gen_gaussian_lin_mono(mu, sigma, n):
    x = np.random.normal(mu, sigma, n)
    z = np.random.normal(mu, sigma, n)
    y = 0.009 * x + 0.021 * z
    return x, y

def gen_notgaussian_notlin_mono(mu, sigma, n):
    x = np.random.normal(mu, sigma, n)
    z = np.random.normal(mu, sigma, n)
    y = np.exp(0.12*x) + 0.09*x + z
    return x, y

def gen_notgaussian_notlin_notmono(mu, sigma, n):
    x = np.random.normal(mu, sigma, n)
    z = np.random.normal(mu, sigma, n)
    y = 0.00001 *np.sin(x)  + 0.01*np.sin(z)
    return x, z

## X и Y гауссовские

In [313]:
num_experiments = 1000
n = 100
mu = 5
sigma = 10

Corr_test_frequency_res = []
Spearman_test_frequency_res  = []
Pirson_test_frequency_res  = []

In [314]:
c, s, p = beta_error_frequency(mu, sigma, n, num_experiments, f = gen_gaussian_lin_mono)

In [315]:
c,s,p

(0.032, 0.051, 0.598)

## X и Y не гауссовские, монотонная зависимость

In [395]:
num_experiments = 1000
n = 100
mu = 5
sigma = 10

Corr_test_frequency_res = []
Spearman_test_frequency_res  = []
Pirson_test_frequency_res  = []

In [396]:
c, s, p = beta_error_frequency(mu, sigma, n, num_experiments, f = gen_notgaussian_notlin_mono)

In [397]:
c,s,p

(0.027, 0.069, 0.286)

## X и Y не гауссовские, немонотонная зависимость

In [387]:
num_experiments = 1000
n = 100
mu = 5
sigma = 10

Corr_test_frequency_res = []
Spearman_test_frequency_res  = []
Pirson_test_frequency_res  = []

In [388]:
c, s, p = beta_error_frequency(mu, sigma, n, num_experiments, f = gen_notgaussian_notlin_notmono)

In [389]:
c,s,p

(0.955, 0.945, 0.59)